In [1]:
# Import Dependencies
import csv
import pandas as pd
import numpy as np
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from keras import regularizers

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Read Raw Dataset
df = pd.read_excel('/content/drive/My Drive/AI_MedProject/raw_data.xlsx')

In [ ]:
df.head()

In [ ]:
# Fill all NaN with the values above
data = df.fillna(method='ffill')

In [ ]:
data.head()

In [ ]:
# Process Disease and Symptom Names
def process_data(data):
    data_list = []
    data_name = data.replace('^','_').split('_')
    n = 1
    for names in data_name:
        if (n % 2 == 0):
            data_list.append(names)
        n += 1
    return data_list

In [ ]:
# Data Cleanup
disease_list = []
disease_symptom_dict = defaultdict(list)
disease_symptom_count = {}
count = 0

for idx, row in data.iterrows():

    # Get the Disease Names
    if (row['Disease'] !="\xc2\xa0") and (row['Disease'] != ""):
        disease = row['Disease']
        disease_list = process_data(data=disease)
        count = row['Count of Disease Occurrence']

    # Get the Symptoms Corresponding to Diseases
    if (row['Symptom'] !="\xc2\xa0") and (row['Symptom'] != ""):
        symptom = row['Symptom']
        symptom_list = process_data(data=symptom)
        for d in disease_list:
            for s in symptom_list:
                disease_symptom_dict[d].append(s)
            disease_symptom_count[d] = count

In [ ]:
# See that the data is Processed Correctly
disease_symptom_dict

In [ ]:
# Count of Disease Occurence w.r.t each Disease
disease_symptom_count

In [ ]:
# Save cleaned data as CSV
f = open('/content/drive/My Drive/AI_MedProject/cleaned_data.csv', 'w')

with f:
    writer = csv.writer(f)
    for key, val in disease_symptom_dict.items():
        for i in range(len(val)):
            writer.writerow([key, val[i], disease_symptom_count[key]])

In [ ]:
# Read Cleaned Data as DF
df = pd.read_csv('/content/drive/My Drive/AI_MedProject/cleaned_data.csv')
df.columns = ['disease', 'symptom', 'occurence_count']
df.head()

In [ ]:
# Remove any rows with empty values
df.replace(float('nan'), np.nan, inplace=True)
df.dropna(inplace=True)

In [ ]:
from sklearn import preprocessing

In [ ]:
n_unique = len(df['symptom'].unique())
n_unique

In [ ]:
df.dtypes

In [ ]:
# Encode the Labels
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df['symptom'])
print(integer_encoded)

In [ ]:
# One Hot Encode the Labels
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

In [ ]:
onehot_encoded[0]

In [ ]:
len(onehot_encoded[0])

In [ ]:
cols = np.asarray(df['symptom'].unique())
cols

In [ ]:
# Create a new dataframe to save OHE labels
df_ohe = pd.DataFrame(columns = cols)
df_ohe.head()

In [ ]:
for i in range(len(onehot_encoded)):
    df_ohe.loc[i] = onehot_encoded[i]

In [ ]:
df_ohe.head()

In [ ]:
len(df_ohe)

In [ ]:
# Disease Dataframe
df_disease = df['disease']
df_disease.head()

In [ ]:
# Concatenate OHE Labels with the Disease Column
df_concat = pd.concat([df_disease,df_ohe], axis=1)
df_concat.head()

In [ ]:
df_concat.drop_duplicates(keep='first',inplace=True)

In [ ]:
df_concat.head()

In [ ]:
len(df_concat)

In [ ]:
cols = df_concat.columns
cols

In [ ]:
cols = cols[1:]

In [ ]:
# Since, every disease has multiple symptoms, combine all symptoms per disease per row
df_concat = df_concat.groupby('disease').sum()
df_concat = df_concat.reset_index()
df_concat[:5]

In [ ]:
len(df_concat)

In [ ]:
df_concat.to_csv("/content/drive/My Drive/AI_MedProject/training_dataset.csv", index=False)

In [ ]:
# One Hot Encoded Features
X = df_concat[cols]

# Labels
y = df_concat['disease']

## Model Training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [ ]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Encode the target variable
y_train_encoded = label_encoder.fit_transform(y_train)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Example input sequences
sequences = [[1, 2, 3], [4, 5], [6, 7, 8, 9]]

# Pad sequences to a maximum length of 200
padded_sequences = pad_sequences(sequences, maxlen=200)

print(padded_sequences)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000  # Replace with the actual vocabulary size
embedding_dim = 100  # Replace with the desired embedding dimension
max_len = 200  # Replace with the maximum sequence length
num_classes = 10  # Replace with the actual number of classes in your problem
epochs = 200

# Example input sequences
your_input_sequences = [[1, 2, 3], [4, 5], [6, 7, 8, 9]]

# Pad sequences to a maximum length of 200
padded_sequences = pad_sequences(your_input_sequences, maxlen=max_len)

# Example training labels
your_training_labels = [0, 1, 0]  # Replace with your actual training labels

# Convert training labels to NumPy array
training_labels = np.array(your_training_labels)

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(LSTM(110, return_sequences=True))
model.add(LSTM(110))
model.add(Dense(208, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model on the training set with validation split
history = model.fit(padded_sequences, training_labels, epochs=epochs, validation_split=0.2)

# Calculate validation accuracy manually
validation_accuracy = model.evaluate(padded_sequences, training_labels)[1]

# Plot the training and validation loss
plt.plot(range(1, epochs + 1), history.history['loss'], label='Training Loss')
plt.plot(range(1, epochs + 1), history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.plot(range(1, epochs + 1), history.history['accuracy'], label='Training Accuracy')
plt.plot(range(1, epochs + 1), history.history['val_accuracy'], label='Validation Accuracy')
plt.axhline(y=validation_accuracy, color='r', linestyle='--', label='Final Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
len(X_test), len(y_test)

In [ ]:
dt = DecisionTreeClassifier()
clf_dt=dt.fit(X, y)

In [ ]:
clf_dt.score(X, y)

In [ ]:
disease_pred = clf_dt.predict(X)

In [ ]:
disease_real = y.values

In [ ]:
for i in range(0, len(disease_real)):
    if disease_pred[i]!=disease_real[i]:
        print ('Pred: {0}\nActual: {1}\n'.format(disease_pred[i], disease_real[i]))

In [ ]:
# Save the trained model
model.save("/content/drive/My Drive/AI_MedProject/chat_model")